# Creating/Updating Romancal Parameter Files
**NOTE: Custom parameter files are not needed to run Romancal. This notebook needs only be used by STScI Instrument scientists or other community members creating specialized Roman calibration pipelines utilizing Romancal steps. If this is not you, please do not worry about the contents of this notebook.**

## Parameter files
Each Romancal step/pipeline houses three methods for specifying parameter values. The first is the hard-coded values within the source code upon each class's definition. These are the base values that are used if no other values are specified. The second are user provided values at execution, whether that is within a command-line call or in a python script/notebook. These user provided values override any other specified parameter values to ensure users have the final say.

In between these two options are parameter files. Copies of these asdf files are stored in CRDS and utilize the conditional filtering of CRDS to allow for different step parameters depending on the observation conditions of each individual data product. For example, different parameter files can be specified for imaging or spectroscopy data, or for different detectors to better handle their individual noise properties. These files will primarily be updated in preparation for the data releases to improve the calibration quality of the data. Custom parameter files can also be created and used locally if the files stored in CRDS are not sufficient. For more information about parameter files in Romancal, see [ASDF Parameter Files](https://roman-pipeline.readthedocs.io/en/latest/roman/stpipe/config_asdf.html).

This notebook provides an example of how to modify existing parameter files (the main use case) and how to create a new parameter file (which will likely not be needed unless new steps are added to Romancal). We start with importing the parameter file class from the reference file pipeline and the datatime package to update some dates.

In [ ]:
from wfi_reference_pipeline.romancal_parameter_files.parameter_file import ParameterFile
from datetime import datetime

## Opening/modifying a parameter file
To simplify the updating of parameter files, we have created a `ParameterFile` class in the WFI Reference File Pipeline (RFP). This class takes in the name of an existing parameter file, opens the file, copies any relevant metadata and existing updated parameters as attributes, and ensures all the parameters from the Romancal step are present.

When making a new parameter file, simply input the name of the parameter file (e.g. "pars-rampfitstep", "pars-jumpstep", "pars-exposurepipeline") and the class will initialize using the defaults from the Romancal step itself. 

When the class is initialized, the current set of step-specific parameters are printed for convenience. These values will be the Romancal defaults unless they have been updated in the input file.  

In [ ]:
input_filepath = 'pars-rampfitstep'
par_class = ParameterFile(input_filepath)

Now, update any parameter values desired. The line below is commented out so as not to accidentally update the file when running the notebook.

In [ ]:
#par_class.use_ramp_jump_detection = False
# ...

When updating files, some of the instrument or exposure metadata might need to be updated as well depending on the intended use of the parameter file.

In [ ]:
par_class.meta['detector'] = 'WFI02'

## Saving a parameter new file

Once the class is updated sufficiently, a few metadata fields need to be updated to ensure proper versioning. If these metadata fields are not updated, the new file will not save.

In [ ]:
par_class.meta['author'] = 'test author'
par_class.meta['pedigree'] = 'DUMMY'
par_class.meta['useafter'] = datetime.today().isoformat()
par_class.meta['description'] = 'This is a sample description for the Rampfit Step parameter file.' 
# the actual description should explain what the use of the parameter file is

When the class is ready to save to the asdf file, we can use the `write` method to save the new parameter file. The `write` method requires a new history message that describes what was changed in the update to the file. This is captured by the `hist_message` string that is passed to the method. Additionally, the written file with be a "development" version by default, indicated by a `_dev.asdf` suffix to the file name. To remove this suffix, set `as_dev_file=False`.

In [ ]:
hist_message = 'This is the first default file version'
par_class.write(hist_message, as_dev_file=True)

By default, the filename will include the file suffix and the date of creation (e.g. roman_wfi_pars-rampfitstep_20241011_dev.asdf). A different filename can be used by setting the `output_filepath` keyword arguement. Additionally, the input filepath can be overwritten by setting `use_input_filepath=True`.

**NOTE:** Under the current implementation, only *non-default* parameter values will be saved to the output file. If no parameters have non-default values, an empty `parameters` dictionary will be saved.

## Specifying step parameters within pipeline parameter files

In addition to the pipeline's specific parameters, each pipeline parameter file can also contain step-specific paramters under the `steps` list. This allows for unique step parameters to be utilized for different instances of the pipeline. Any step parameter specified within the pipeline parameter file will overwrite any parameter specified within the step-specific parameter file.

**In practice, RTB only plans to use the step definitions within the pipeline parameter files to force a given pipeline to skip steps. All other step parameters will be governed by the individual step parameter files.**

Below is an example of what this could look like:

In [ ]:
input_filepath = 'pars-exposurepipeline'
par_class = ParameterFile(input_filepath)

To see the list of steps that are used by the given pipeline, we can print the `step_names` attribute.

In [ ]:
par_class.step_names

Each step is saved as an attribute in the `ParameterFile` class using its name as the attribute name. The steps are dictionaries with `class`, `name`, and `parameters` as the keys. To update a step parameter, simply update the parameter within the step's `parameters` key.

In [ ]:
par_class.flatfield['parameters']['skip'] = True
par_class.meta['exposure']['type'] = 'WFI_GRISM'

After modifying the parameters, follow the same saving instructions as above (including modifying the required metadata and history entry). Similar to saving step parameter files, any parameters that are not modified from the default values will not be included in the file.